# Utility functionality for reading and plotting results

### References : TODO

### Imports

In [2]:
import os
import pandas as pd
import sys
from matplotlib import pyplot as plt

## Settings

In [8]:
# ------ Settings ------
# => These variables should be set to the desired settings!

# The files in the directory should have names of the form "{dataset}-factor-{downsample factor}-{any remaining information}"
directory = ''

dataset = 'AirChange' # The dataset

# The algorithms to consider
algorithms = [
    'Change Vector Analysis',
    'Principal Component Differential Analysis',
    'Image Differencing',
    'Image Ratioing',
    'Image Regression',
    'Iterative Reweighted MAD',
    'Multivariate Alteration Detection',
    'Change Detection PCA k-Means',
    'Principal Component Differential Analysis'
]

# The metrics to consider
metrics = [
    'OA',
    'UA',
    'Recall',
    'FMeasure',
    'AUC',
    'Kappa'
]

plot_hist = False
plot_mean = True
plot_scatter = True

In [ ]:
data_per_alg = {}
factors = []

for alg in algorithms:
    data_per_alg[alg] = {}

# Read in all the data
for fname in os.listdir(directory):
    fpath = os.path.join(directory, fname)
    ftype = os.path.splitext(fname)[-1].lower()

    # Check if it is an Excel file
    if os.path.isfile(fpath) and ftype == '.xls':
        df = pd.read_excel(fpath, header=[0])

        factor = fname.split('-')[2]
        factors.append(factor)

        for alg in algorithms:
            data_per_alg[alg][factor] = df.loc[df['name'] == alg]

# Create all needed directories
plt_dir = os.path.join(directory, 'Plots')
if not os.path.exists(plt_dir):
    os.makedirs(plt_dir)

hist_dir = os.path.join(plt_dir, 'Histograms')
if not os.path.exists(hist_dir):
    os.makedirs(hist_dir)

means_dir = os.path.join(plt_dir, 'Means')
if not os.path.exists(means_dir):
    os.makedirs(means_dir)

scatter_dir = os.path.join(plt_dir, 'Scatter')
if not os.path.exists(scatter_dir):
    os.makedirs(scatter_dir)

for metric in metrics:
    if not os.path.exists(os.path.join(means_dir, metric)):
        os.makedirs(os.path.join(means_dir, metric))

    if not os.path.exists(os.path.join(scatter_dir, metric)):
        os.makedirs(os.path.join(scatter_dir, metric))

factors_ints = [int(factor) for factor in factors]
for alg in algorithms:
    for metric in metrics:
        x_axs_plot_mean = factors_ints
        y_axs_plot_mean = []

        x_axs_scatter = []
        y_axs_scatter = []
        
        for factor in factors:
            m_res = data_per_alg[alg][factor].loc[:, [metric]][metric]

            y_axs_plot_mean.append(m_res.mean())

            x_axs_scatter.extend([int(factor) for _ in m_res])
            y_axs_scatter.extend(m_res)

            # Plot histogram
            if plot_hist:
                plt.clf()
                plt.title(dataset + ', ' + alg + ', ' + metric)
                plt.hist(m_res, 6)
                plt.savefig(os.path.join(hist_dir, alg + '-' + metric + '-' + factor + '.png'))

        # Plot means
        if plot_mean:
            plt.clf()
            plt.title(dataset + ', ' + alg + ', ' + metric)
            plt.plot(x_axs_plot_mean, y_axs_plot_mean, 'go-')
            plt.xticks(factors_ints, factors)
            plt.savefig(os.path.join(means_dir, metric, alg + '-' + metric + '-' + factor + '.png'))

        # Plot scatter
        if plot_scatter:
            plt.clf()
            plt.title(dataset + ', ' + alg + ', ' + metric)
            plt.scatter(x_axs_scatter, y_axs_scatter, s=10, color='g')
            plt.xticks(factors_ints, factors)
            plt.savefig(os.path.join(scatter_dir, metric, alg + '-' + metric + '-' + factor + '.png'))
